In [33]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn import metrics

import statsmodels.api as sm
from statsmodels.tools import eval_measures

In [34]:
data=pd.read_csv('./properatti.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121220 entries, 0 to 121219
Data columns (total 26 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Unnamed: 0                  121220 non-null  int64  
 1   operation                   121220 non-null  object 
 2   property_type               121220 non-null  object 
 3   place_name                  121197 non-null  object 
 4   place_with_parent_names     121220 non-null  object 
 5   country_name                121220 non-null  object 
 6   state_name                  121220 non-null  object 
 7   geonames_id                 102503 non-null  float64
 8   lat-lon                     69670 non-null   object 
 9   lat                         69670 non-null   float64
 10  lon                         69670 non-null   float64
 11  price                       100810 non-null  float64
 12  currency                    100809 non-null  object 
 13  price_aprox_lo

In [35]:
capital_federal_mask = data.state_name=="Capital Federal"
capital_federal = data[capital_federal_mask]
capital_federal

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...
7,7,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,...,40.0,3066.666667,3450.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bot_venta_depart...,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,JOSE HERNANDEZ 1400 MONOAMBIENTE ESTRENAR CAT...,https://thumbs4.properati.com/1/IHxARynlr8sPEW...
8,8,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,...,60.0,3000.000000,3250.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bou_venta_depart...,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...,"JOSE HERNANDEZ 1400 DOS AMBIENTES ESTRENAR ,...",https://thumbs4.properati.com/2/J3zOjgaFHrkvnv...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121160,121160,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.563685346,-58.4426825896",-34.563685,...,NaN,2611.464968,NaN,NaN,NaN,6300.0,http://www.properati.com.ar/1cj5v_venta_depart...,Edificio en esquina de muy buena categoría de ...,Av .Luis Maria Campos 1400- Belgrano-Capital F...,https://thumbs4.properati.com/0/zeLNBuyylOJL7z...
121209,121209,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.563685346,-58.4426825896",-34.563685,...,NaN,2611.464968,NaN,NaN,NaN,6300.0,http://www.properati.com.ar/1cj7c_venta_depart...,Edificio en esquina de muy buena categoría de ...,Av .Luis Maria Campos 1400- Belgrano-Capital F...,https://thumbs4.properati.com/0/zeLNBuyylOJL7z...
121215,121215,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,NaN,NaN,...,93.0,7699.115044,9354.838710,NaN,NaN,10000.0,http://www.properati.com.ar/1cja2_venta_depart...,TORRE FORUM ALCORTA - MÁXIMA CATEGORÍA.Impecab...,Torre Forum Alcorta- Impecable 3 ambientes,https://thumbs4.properati.com/1/bjms0KnaAnlNoQ...
121217,121217,sell,apartment,Villa Urquiza,|Argentina|Capital Federal|Villa Urquiza|,Argentina,Capital Federal,3433775.0,"-34.5706388726,-58.4755963355",-34.570639,...,39.0,2858.695652,3371.794872,NaN,NaN,NaN,http://www.properati.com.ar/1cja7_venta_depart...,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,https://thumbs4.properati.com/9/YAe_-2gRVykADP...


In [36]:
capital_federal_dos = capital_federal.loc[:,('property_type','place_name','surface_total_in_m2','surface_covered_in_m2', 'price')]
capital_federal_dos

,property_type,place_name,surface_total_in_m2,surface_covered_in_m2,price
0,PH,Mataderos,55.0,40.0,62000.0
2,apartment,Mataderos,55.0,55.0,72000.0
3,PH,Liniers,NaN,NaN,95000.0
7,apartment,Belgrano,45.0,40.0,138000.0
8,apartment,Belgrano,65.0,60.0,195000.0
...,...,...,...,...,...
121160,apartment,Belgrano,157.0,NaN,410000.0
121209,apartment,Belgrano,157.0,NaN,410000.0
121215,apartment,Belgrano,113.0,93.0,870000.0
121217,apartment,Villa Urquiza,46.0,39.0,131500.0


In [37]:
capital_federal_apartment_mask = capital_federal.property_type=='apartment'
capital_federal_apartment = capital_federal_dos[capital_federal_apartment_mask]
capital_federal_apartment

,property_type,place_name,surface_total_in_m2,surface_covered_in_m2,price
2,apartment,Mataderos,55.0,55.0,72000.0
7,apartment,Belgrano,45.0,40.0,138000.0
8,apartment,Belgrano,65.0,60.0,195000.0
13,apartment,Palermo Soho,50.0,30.0,111700.0
14,apartment,Palermo Soho,42.0,31.0,147900.0
...,...,...,...,...,...
121160,apartment,Belgrano,157.0,NaN,410000.0
121209,apartment,Belgrano,157.0,NaN,410000.0
121215,apartment,Belgrano,113.0,93.0,870000.0
121217,apartment,Villa Urquiza,46.0,39.0,131500.0


In [38]:
#Porcentaje de nulls respecto al total del dataset por columna

percent_missing = capital_federal_apartment.isnull().sum() * 100 / len(capital_federal_apartment)
missing_value_df = pd.DataFrame({'column_name': capital_federal_apartment.columns,
                                 'percent_missing': percent_missing})

missing_value_df.sort_values('percent_missing', inplace=True, ascending=False)
missing_value_df

,column_name,percent_missing
surface_total_in_m2,surface_total_in_m2,16.484817
price,price,10.663165
surface_covered_in_m2,surface_covered_in_m2,6.646447
property_type,property_type,0.000000
place_name,place_name,0.000000


In [39]:
capital_federal_apartment.surface_total_in_m2.fillna(capital_federal_apartment.surface_covered_in_m2, inplace=True)


C:\Users\Juan\AppData\Local\Temp\ipykernel_18232\3128688942.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  capital_federal_apartment.surface_total_in_m2.fillna(capital_federal_apartment.surface_covered_in_m2, inplace=True)


In [40]:
capital_federal_apartment['surface_uncovered_in_m2'] = capital_federal_apartment.surface_total_in_m2 - capital_federal_apartment.surface_covered_in_m2
capital_federal_apartment

C:\Users\Juan\AppData\Local\Temp\ipykernel_18232\3450111080.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  capital_federal_apartment['surface_uncovered_in_m2'] = capital_federal_apartment.surface_total_in_m2 - capital_federal_apartment.surface_covered_in_m2


,property_type,place_name,surface_total_in_m2,surface_covered_in_m2,price,surface_uncovered_in_m2
2,apartment,Mataderos,55.0,55.0,72000.0,0.0
7,apartment,Belgrano,45.0,40.0,138000.0,5.0
8,apartment,Belgrano,65.0,60.0,195000.0,5.0
13,apartment,Palermo Soho,50.0,30.0,111700.0,20.0
14,apartment,Palermo Soho,42.0,31.0,147900.0,11.0
...,...,...,...,...,...,...
121160,apartment,Belgrano,157.0,NaN,410000.0,NaN
121209,apartment,Belgrano,157.0,NaN,410000.0,NaN
121215,apartment,Belgrano,113.0,93.0,870000.0,20.0
121217,apartment,Villa Urquiza,46.0,39.0,131500.0,7.0


In [41]:
capital_federal_apartment.drop('surface_total_in_m2', axis=1, inplace=True)


C:\Users\Juan\AppData\Local\Temp\ipykernel_18232\573191145.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  capital_federal_apartment.drop('surface_total_in_m2', axis=1, inplace=True)


In [42]:
percent_missing = capital_federal_apartment.isnull().sum() * 100 / len(capital_federal_apartment)
missing_value_df = pd.DataFrame({'column_name': capital_federal_apartment.columns,
                                 'percent_missing': percent_missing})

missing_value_df.sort_values('percent_missing', inplace=True, ascending=False)
missing_value_df

,column_name,percent_missing
price,price,10.663165
surface_covered_in_m2,surface_covered_in_m2,6.646447
surface_uncovered_in_m2,surface_uncovered_in_m2,6.646447
property_type,property_type,0.000000
place_name,place_name,0.000000


In [43]:
capital_federal_apartment_clean = capital_federal_apartment.dropna()
capital_federal_apartment_clean

,property_type,place_name,surface_covered_in_m2,price,surface_uncovered_in_m2
2,apartment,Mataderos,55.0,72000.0,0.0
7,apartment,Belgrano,40.0,138000.0,5.0
8,apartment,Belgrano,60.0,195000.0,5.0
13,apartment,Palermo Soho,30.0,111700.0,20.0
14,apartment,Palermo Soho,31.0,147900.0,11.0
...,...,...,...,...,...
121157,apartment,Belgrano,35.0,128000.0,3.0
121158,apartment,Recoleta,39.0,165000.0,5.0
121215,apartment,Belgrano,93.0,870000.0,20.0
121217,apartment,Villa Urquiza,39.0,131500.0,7.0


In [44]:
percent_missing = capital_federal_apartment_clean.isnull().sum() * 100 / len(capital_federal_apartment_clean)
missing_value_df = pd.DataFrame({'column_name': capital_federal_apartment_clean.columns,
                                 'percent_missing': percent_missing})

missing_value_df.sort_values('percent_missing', inplace=True, ascending=False)
missing_value_df

,column_name,percent_missing
property_type,property_type,0.0
place_name,place_name,0.0
surface_covered_in_m2,surface_covered_in_m2,0.0
price,price,0.0
surface_uncovered_in_m2,surface_uncovered_in_m2,0.0


In [45]:
capital_federal_apartment_clean.drop('property_type', inplace=True, axis=1)
capital_federal_apartment_clean

C:\Users\Juan\AppData\Local\Temp\ipykernel_18232\3058871001.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  capital_federal_apartment_clean.drop('property_type', inplace=True, axis=1)


,place_name,surface_covered_in_m2,price,surface_uncovered_in_m2
2,Mataderos,55.0,72000.0,0.0
7,Belgrano,40.0,138000.0,5.0
8,Belgrano,60.0,195000.0,5.0
13,Palermo Soho,30.0,111700.0,20.0
14,Palermo Soho,31.0,147900.0,11.0
...,...,...,...,...
121157,Belgrano,35.0,128000.0,3.0
121158,Recoleta,39.0,165000.0,5.0
121215,Belgrano,93.0,870000.0,20.0
121217,Villa Urquiza,39.0,131500.0,7.0


In [46]:
dummies = pd.get_dummies(capital_federal_apartment_clean['place_name']).rename(columns=lambda x: 'Barrio_' + str(x))
dummies

# X = pd.concat([dummies, std_df], axis = 1)

,Barrio_Abasto,Barrio_Agronomía,Barrio_Almagro,Barrio_Balvanera,Barrio_Barracas,Barrio_Barrio Norte,Barrio_Belgrano,Barrio_Boca,Barrio_Boedo,Barrio_Caballito,...,Barrio_Villa Lugano,Barrio_Villa Luro,Barrio_Villa Ortuzar,Barrio_Villa Pueyrredón,Barrio_Villa Real,Barrio_Villa Riachuelo,Barrio_Villa Santa Rita,Barrio_Villa Soldati,Barrio_Villa Urquiza,Barrio_Villa del Parque
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121157,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
121158,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
121215,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
121217,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [47]:
numericals = ['surface_uncovered_in_m2', 'surface_covered_in_m2']

X = capital_federal_apartment_clean[numericals]

scaler = StandardScaler()
scaler.fit(X)

std_numerical_data = scaler.transform(X)
std_df = pd.DataFrame(std_numerical_data, index=capital_federal_apartment_clean.index)
std_df.columns = [i + '_std' for i in numericals]

std_df
#data_scaled = pd.DataFrame(scaled_features, index=df.index, columns=df.columns)


,surface_uncovered_in_m2_std,surface_covered_in_m2_std
2,-0.251288,-0.020559
7,-0.089812,-0.032630
8,-0.089812,-0.016536
13,0.394614,-0.040677
14,0.103958,-0.039873
...,...,...
121157,-0.154402,-0.036654
121158,-0.089812,-0.033435
121215,0.394614,0.010020
121217,-0.025222,-0.033435


In [48]:
X = pd.concat([dummies, std_df], axis = 1)
X

,Barrio_Abasto,Barrio_Agronomía,Barrio_Almagro,Barrio_Balvanera,Barrio_Barracas,Barrio_Barrio Norte,Barrio_Belgrano,Barrio_Boca,Barrio_Boedo,Barrio_Caballito,...,Barrio_Villa Ortuzar,Barrio_Villa Pueyrredón,Barrio_Villa Real,Barrio_Villa Riachuelo,Barrio_Villa Santa Rita,Barrio_Villa Soldati,Barrio_Villa Urquiza,Barrio_Villa del Parque,surface_uncovered_in_m2_std,surface_covered_in_m2_std
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-0.251288,-0.020559
7,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,-0.089812,-0.032630
8,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,-0.089812,-0.016536
13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.394614,-0.040677
14,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.103958,-0.039873
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121157,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,-0.154402,-0.036654
121158,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-0.089812,-0.033435
121215,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0.394614,0.010020
121217,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,-0.025222,-0.033435


In [49]:
y = capital_federal_apartment_clean.price

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 117)


In [51]:
# Tenemos que agregar explícitamente a una constante:
X_train_sm = sm.add_constant(X_train)

model = sm.OLS(y_train, X_train_sm).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.084
Model:                            OLS   Adj. R-squared:                  0.081
Method:                 Least Squares   F-statistic:                     23.72
Date:                Tue, 29 Nov 2022   Prob (F-statistic):          1.09e-253
Time:                        12:49:06   Log-Likelihood:            -2.4260e+05
No. Observations:               16016   AIC:                         4.853e+05
Df Residuals:                   15953   BIC:                         4.858e+05
Df Model:                          62                                         
Covariance Type:            nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const                        2.831e+05   2.29e+04     12.358      0.000    2.38e+05    3.28e+05
Barrio_Abasto               -1.165e+05   1.26e+05     -0.923      0.356   -3.64e+05    1.31e+05
Barrio_Agronomía            -1.447e+05    2.2e+05     -0.657      0.511   -5.77e+05    2.87e+05
Barrio_Almagro               -5.82e+04   4.38e+04     -1.327      0.184   -1.44e+05    2.77e+04
Barrio_Balvanera            -1.217e+05   5.15e+04     -2.361      0.018   -2.23e+05   -2.07e+04
Barrio_Barracas             -1.326e+05   6.83e+04     -1.940      0.052   -2.67e+05    1349.182
Barrio_Barrio Norte          2.863e+04   4.24e+04      0.675      0.500   -5.45e+04    1.12e+05
Barrio_Belgrano              3.457e+05   3.25e+04     10.639      0.000    2.82e+05    4.09e+05
Barrio_Boca                  3.423e+05   8.56e+04      3.999      0.000    1.75e+05     5.1e+05
Barrio_Boedo                 1.853e+05   4.55e+04      4.076      0.000    9.62e+04    2.74e+05
Barrio_Caballito            -1.008e+05   3.41e+04     -2.953      0.003   -1.68e+05   -3.39e+04
Barrio_Capital Federal      -4.028e+04   5.77e+04     -0.698      0.485   -1.53e+05    7.29e+04
Barrio_Catalinas             1.944e+05   9.04e+05      0.215      0.830   -1.58e+06    1.97e+06
Barrio_Centro / Microcentro -1.169e+05    8.8e+04     -1.328      0.184   -2.89e+05    5.56e+04
Barrio_Chacarita            -4.673e+04    8.8e+04     -0.531      0.595   -2.19e+05    1.26e+05
Barrio_Coghlan              -1319.7650   8.13e+04     -0.016      0.987   -1.61e+05    1.58e+05
Barrio_Colegiales            1.418e+05   6.98e+04      2.033      0.042    5084.431    2.79e+05
Barrio_Congreso             -1.105e+05   7.17e+04     -1.542      0.123   -2.51e+05       3e+04
Barrio_Constitución         -4.796e+04   8.99e+04     -0.533      0.594   -2.24e+05    1.28e+05
Barrio_Flores               -2.911e+04   4.11e+04     -0.709      0.478    -1.1e+05    5.14e+04
Barrio_Floresta             -1.399e+05    6.9e+04     -2.029      0.043   -2.75e+05   -4715.816
Barrio_Las Cañitas          -3799.4664   1.07e+05     -0.036      0.972   -2.13e+05    2.06e+05
Barrio_Liniers              -1.707e+05    9.6e+04     -1.779      0.075   -3.59e+05    1.74e+04
Barrio_Mataderos            -1.843e+05   1.13e+05     -1.635      0.102   -4.05e+05    3.67e+04
Barrio_Monserrat            -1.198e+05   5.68e+04     -2.108      0.035   -2.31e+05   -8407.062
Barrio_Monte Castro         -1.713e+05   1.82e+05     -0.940      0.347   -5.28e+05    1.86e+05
Barrio_Nuñez                   2.4e+05   5.05e+04      4.750      0.000    1.41e+05    3.39e+05
Barrio_Once                 -1.643e+04   9.85e+04     -0.167      0.868   -2.09e+05    1.77e+05
Barrio_Palermo               1.221e+05   3.28e+04      3.721      0.000    5.78e+04    1.86e+05
Barrio_Pale